![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)
# G-Research Crypto - Modélisation (FR)

Dans le concours de prédiction G-Research Crypto, les participants ont le défi de prédire les rendements des prix sur un ensemble de crypto-monnaies majeures.

Références utilisées : 
* [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)
* [G-Research Crypto - Starter XGB Pipeline](https://www.kaggle.com/tarlannazarov/g-research-crypto-starter-xgb-pipeline)
* [LGDM model with new features better generalization](https://www.kaggle.com/swaralipibose/lgdm-model-with-new-features-better-generalization)

---
# Chargement

## Libraries et Fonctions

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import preprocessing, linear_model, pipeline
from lightgbm import LGBMRegressor

In [ ]:
stats.pearsonr([1,20,1],[-6,-5,-6])[0]

In [ ]:
def reduce_mem_usage(df):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def ponderation(corrs):
    """Moyenne pondérée des corrélations positives.
    """
    tmp = (df_asset_details['Weight'] * corrs)
    return tmp[tmp > 0].sum() / df_asset_details['Weight'].sum()
#     return (df_asset_details['Weight'] * corrs).abs().sum() / df_asset_details['Weight'].sum()

def plot_feature_importance(feature_names, model):
    fi_df = pd.DataFrame()
    fi_df['features'] = feature_names
    if type(model) == xgb.XGBRegressor:
        fi_df['importance'] = model.feature_importances_
    elif type(model) == LGBMRegressor:
        fi_df['importance'] = model.booster_.feature_importance(importance_type="gain")
    else:
        print('Pas de feature_importances_ configuré pour ce modèle.')
        return None

    fig, ax = plt.subplots(1, 1, figsize=(15, 7))
    sns.barplot(y='importance', x='features', data=fi_df.sort_values(by=['importance'], ascending=False), ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')
    plt.show()
    
def RSI(close: pd.DataFrame, period: int = 14) -> pd.Series:
    # https://gist.github.com/jmoz/1f93b264650376131ed65875782df386
    """See source https://github.com/peerchemist/finta
    and fix https://www.tradingview.com/wiki/Talk:Relative_Strength_Index_(RSI)
    Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements.
    RSI oscillates between zero and 100. Traditionally, and according to Wilder, RSI is considered overbought when above 70 and oversold when below 30.
    Signals can also be generated by looking for divergences, failure swings and centerline crossovers.
    RSI can also be used to identify the general trend.
    """
    delta = close.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    _gain = up.ewm(com=(period - 1), min_periods=period).mean()
    _loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()
    RS = _gain / _loss
    return pd.Series(100 - (100 / (1 + RS)))
    
def plot_corrs(c_train, c_test):    
    pd.DataFrame(data={
        f'Entrainement (avg:{ponderation(c_train):.2f})': c_train, 
        f'Test (avg:{ponderation(c_test):.2f})': c_test
    }).plot.bar(figsize=(10,5))
    plt.xlabel('Crypto-monnaies')
    plt.ylabel('Corrélation de Pearson')
    plt.plot([0,13],[0,0], '--k')
    plt.xticks(range(14), df_asset_details.Asset_Name.values, rotation=30, ha='right')
    plt.show()

## Données

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
df = pd.read_csv(data_folder + 'train.csv').pipe(reduce_mem_usage)
df = df.sort_values(by=['timestamp','Asset_ID'])
df = df.iloc[:-16*14]  # Suppression des 16 dernières minutes sans Target
df_train = df[df.timestamp < 1623542400]  # Avant '2021-06-13 00:00:00'
df_asset_details = pd.read_csv(data_folder + 'asset_details.csv').sort_values("Asset_ID")

In [ ]:
IDs = df_asset_details.Asset_ID.values
tmin, tmax = df_train['timestamp'].min(), df_train['timestamp'].max()
df_train = df_train.set_index('timestamp')
df_tmp = pd.DataFrame()
for i in IDs:
    sub_reindexed = df_train.loc[df_train['Asset_ID'] == i].reindex(range(tmin, tmax+60, 60), method='bfill')
    df_tmp = pd.concat([df_tmp, sub_reindexed], axis=0)
    if i == 1:
        df_btc = sub_reindexed
        
df_train = df_tmp.reset_index()
df_btc = df_btc.reset_index()

---
# Préparation

## Cibles de prédiction et évaluation
Ce concours de prévision vise à prédire les rendements dans un futur proche des prix $P^a$, pour chaque actif $a$.
Pour chaque ligne de l'ensemble de données, nous incluons la cible de prédiction, `Target`.
La cible est dérivée des **log return** ($R^a$) sur **15 minutes** :
$$R^a(t)=\log(P^a(t+16) / P^a(t+1))$$

Les rendements des actifs cryptographiques sont fortement corrélés, suivant dans une large mesure le marché global de la cryptographie.
Comme on nous pousse à tester notre capacité à prédire les rendements d'actifs individuels,  une résidualisation linéaire a été effectuée, en supprimant le signal de marché des rendements d'actifs individuels lors de la création de `Target`. 
Plus en détail, si $M(t)$ est la moyenne pondérée des rendements du marché, la cible est :
$$M(t) = \frac{\sum_{a}{w^aR^a(t))}}{\sum_{a}{w^a}}$$
$$\beta^a = \frac{⟨M⋅R^a⟩}{⟨M^2⟩}$$
$$\text{Target}^a(t) = R^a(t)−\beta^aM(t)$$
où la parenthèse $⟨.⟩$ représente la **moyenne mobile** au fil du temps (fenêtres de 3750 minutes), et les mêmes pondérations d'actifs ont été utilisées pour la métrique d'évaluation.

Certaines lignes ont des valeurs nulles pour les cibles en raison de valeurs manquantes dans les prix futurs. 
Les lignes avec des valeurs NULL dans l'ensemble de test sont ignorées à des fins de notation.

Dans la compétition, les prédictions seront évaluées sur une version pondérée du coefficient de corrélation de Pearson, avec des pondérations données par la colonne `Weight` dans le fichier `asset_details.csv`.

## Feature design
Avant de chercher des caractéristiques, il y a quelques mises en garde à prendre en compte :
* pas de fuite : nous ne pouvons pas utiliser une fonctionnalité qui utilise les informations futures (il s'agit d'une tâche de prévision)
* caractéristiques stationnaires : les caractéristiques doivent fonctionner à tout moment (les échelles doivent être stationnaires sur des périodes de temps)

Quelques caractéristiques pertinentes potentielles :
* ...

### Indicateurs techniques

In [ ]:
def get_features(df_asset, df_btc):
    df = df_asset.copy()
    
#     df['price_change'] = df['Close'] / df['Open']
#     df['log_price_change'] = np.log1p(df['price_change'])
#     df['trade'] = df['Close'] - df['Open']
#     df['candle_len'] = np.abs(df['trade'])
    df['gtrade'] = (df['Close'] - df['Open']) / (df['Count'] + 1)
    df['mean_trade'] = df['Volume'] / (df['Count'] + 1)
    
    df['high2low'] = df['High'] / df['Low']
    df['spread'] = df['High'] - df['Low']
    median_price = df[['Open', 'High', 'Low', 'Close']].median(axis=1)
    df['high2median'] = df['High'] / median_price
    df['low2median'] = df['Low'] / median_price
    
    # df['upper_shadow'] = df['High'] / df[['Close', 'Open']].max(axis=1)
    df['upper_shadow'] = df['High'] - df[['Close', 'Open']].max(axis=1)
    # df['lower_shadow'] = df[['Close', 'Open']].min(axis=1) / df['Low']
    df['lower_shadow'] = df[['Close', 'Open']].min(axis=1) - df['Low']
#     df['upper_shadow_log'] = np.log1p(df['upper_shadow'])
#     df['lower_shadow_log'] = np.log1p(df['lower_shadow'])
    
    times = pd.to_datetime(df['timestamp'], unit="s", infer_datetime_format=True)
    df["dayofweek"] = times.dt.dayofweek 
    df["RSI_14"] = RSI(df["Close"], 14)
    
    df["dCloseBTC"] = df_btc.loc[df_btc['timestamp'].isin(df['timestamp']), 'Close'].diff().values
#     df['d2CloseBTC'] = df["dCloseBTC"].diff()
    for t in [3, 5, 15, 30, 60]:
        # Lags
        df[f"dCloseBTC_t-{t}"] = df["dCloseBTC"].shift(t)
#         df[f"d2CloseBTC_t-{t}"] = df["d2CloseBTC"].shift(t)
        
        # Aggrégations
        df[f"mdClose_t-{t}"] = df["Close"].diff().rolling(t).mean()
    
    df = df.drop(columns=['timestamp', 'Asset_ID'])
    return df

### Découpage
Il faut construire des Folds (sous échantillons) indépendants. 
Pour éviter de complexifier la tache avec des DataFrame différentes, on passera par des listes d'index.

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# a = list(range(42))
# list(TimeSeriesSplit(n_splits=3,).split(a))

In [ ]:
# time_ids = df_train["timestamp"].unique()
# ntimes = len(time_ids)

# n_fold = 5
# splits = 0.7

# # 1 mois d'embargo
# embargo_train_test = 60*24*30
# embargo_fold = 60*24*30

# time_per_fold = (ntimes - 5*embargo_train_test - 5*embargo_fold)/5
# train_len = splits*time_per_fold 
# test_len = (1-splits)*time_per_fold

# fold_start = [np.int(i*(ntimes+1)/5) for i in range(6)]

# for i in range(n_fold):
#     time_folds = time_ids[fold_start[i]:fold_start[i+1]-1]
#     df_fold = train_df[train_df.timestamp.isin(time_folds)]
#     df_fold.to_parquet('df_fold_'+str(i)+'.parquet')
    
# del train_df

## Modèles

In [ ]:
template_lin = (
    linear_model.LinearRegression,
    {})

In [ ]:
template_lgbm = (
    LGBMRegressor,
    {
        'n_estimators': 1000,
        'learning_rate': 0.09,
        # 'metric': 'rmse',
        'num_leaves': 500,
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'max_depth': 10,
        # 'subsample': 0.72,
        # 'subsample_freq': 4,
        # 'feature_fraction': 0.4,
        # 'lambda_l1': 1,
        # 'lambda_l2': 1,
        # 'seed': 46,
    })

In [ ]:
template_xgb = (
    xgb.XGBRegressor,
    {
        'n_estimators': 1000,
        'learning_rate': 0.09,
        'max_depth': 10,
        # 'subsample': 0.9,
        # 'colsample_bytree': 0.7,
        # 'colsample_bylevel': 0.75, 
#         'missing': -999,
#         'random_state': 1111,
        'tree_method': 'gpu_hist',
    })

---
# Modélisation

In [ ]:
def get_Xy_and_model_for_asset(template_model, df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]#[-14*13000:]
    
    df_proc = get_features(df.drop(columns='Target'), df_btc)#[-14*13000:])
    df_proc['y'] = df['Target']
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan).dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    pipe = pipeline.Pipeline(steps=[
        ('scaler', preprocessing.StandardScaler()),
        ('model', template_model[0](**template_model[1]))])
    pipe.fit(X, y)
    lastX = df.iloc[-70:].drop(columns='Target')
    
    # Prédiction sur les 2000 derniers éléments pour accélérer le processus
    y_pred = pipe.predict(X[-2000:])
    return lastX, pipe, y[-2000:], y_pred

def procedure(template_model):
    models = {}
    lastXs = {}
    corrs = []
    for i, (asset_id, asset_weight, asset_name) in df_asset_details.iterrows():
        lastX, model, y, y_pred = get_Xy_and_model_for_asset(template_model, df_train, asset_id)    
        corr = stats.pearsonr(y, y_pred)[0]
        models[asset_id] = model
        lastXs[asset_id] = lastX
        corrs.append(corr)
        print(f"Entrainement du modèle for {asset_name:<16} (ID={asset_id:>2}): {corr:5.2f}")
    print(f"{ponderation(corrs):5.2f} <-- Moyenne pondérée des corrélations absolues")
    return models, lastXs, corrs

def test(models):
    import warnings
    warnings.filterwarnings("ignore")

    Xs = dict(lastXs)
    timestamps = df_test.iloc[100*14:]['timestamp'].unique()
    for timestamp in timestamps: 
        if not (timestamp % (6*len(timestamps))):
            print(f"{1+ (timestamp-timestamps[0]) // (6*len(timestamps)):>2} / 10")

        t_test = df_test.loc[df_test['timestamp'] == timestamp]
        for j , row in t_test.iterrows():
            ID = row['Asset_ID']
            model = models[ID]
            Xs[ID] = Xs[ID].shift(-1)
            Xs[ID].iloc[-1] = row[:-2]
            
            x_test = get_features(Xs[ID], df_btc_test)
            y_pred = model.predict(x_test[-1:])[0]

            df_test.loc[row.name, 'Pred'] = y_pred
        
    corrs = []
    for i, row in df_asset_details.iterrows():
        ID = row['Asset_ID']
        corr = stats.pearsonr(
            df_test[100*14:].loc[df_test[100*14:]['Asset_ID'] == ID, 'Target'].values, 
            df_test[100*14:].loc[df_test[100*14:]['Asset_ID'] == ID, 'Pred'].values
        )[0]
        print(f"{corr:5.2f}  {row['Asset_Name']}")
        corrs.append(corr)
    print(f"{ponderation(corrs):5.2f} <-- Moyenne pondérée des corrélations absolues")
    return corrs

def test2(models):
    corrs = []
    for i, (asset_id, asset_weight, asset_name) in df_asset_details.iterrows():
        df = df_test[df_test["Asset_ID"] == asset_id]
        df_proc = get_features(df.drop(columns=['Target','Pred']), df_btc_test)[100:]
        df_proc['y'] = df['Target']
        df_proc = df_proc.replace([np.inf, -np.inf], np.nan).dropna(how="any")

        X = df_proc.drop("y", axis=1)
        y_true = df_proc["y"]
        y_pred = models[asset_id].predict(X)  
          
        corr = stats.pearsonr(y_true, y_pred)[0]
        corrs.append(corr)
        print(f"{asset_name:<16} (ID={asset_id:>2}): {corr:5.2f}")
    print(f"{ponderation(corrs):5.2f} <-- Moyenne pondérée des corrélations absolues")
    return corrs

> 7,54s pour 10 minutes &rarr; 30h pour tout le jeu de test...  
TODO  &rarr; optimiser

## Données de Test manuel

In [ ]:
# Après '2021-06-13 00:00:00' et pour 4 minutes
# df_test = df[df.index >= 1623542400].iloc[:4*14].reset_index().sort_values(by=['timestamp','Asset_ID']).reset_index(drop=True)
# df_test = df[df.timestamp > 1623542400]  # Après '2021-06-13 00:00:00'
df_test = pd.read_csv(data_folder + 'supplemental_train.csv')
df_test = df_test[:-16*14]
# df_test = df_test[:200*14]
df_test = pd.concat([df_train.iloc[-100*14:], df_test])  # Ajout des dernieres valeurs d'entrainement
# df_test = df_test.sort_values(by=['timestamp','Asset_ID'])

tmin, tmax = df_test['timestamp'].min(), df_test['timestamp'].max()
df_test = df_test.set_index('timestamp')
df_btc_test = df_test.loc[df_test['Asset_ID'] == 1].reindex(range(tmin, tmax+60, 60), method='bfill')
df_test = df_test.reset_index()
df_btc_test = df_btc_test.reset_index()
df_test['Pred'] = np.nan

In [ ]:
len(df_btc)/(len(df_btc)+ len(df_btc_test)), len(df_btc_test)/(len(df_btc)+ len(df_btc_test))

## Linéaire

In [ ]:
%%time
models_lin, lastXs, corrs_train_lin = procedure(template_lin)
corrs_test_lin = test(models_lin)

In [ ]:
# Plus rapide mais différent. TODO: Vérifier les fonctions
corrs_test_lin2 = test2(models_lin)

In [ ]:
plot_corrs(corrs_train_lin, corrs_test_lin)

## LGBMRegressor

In [ ]:
%%time
models_lgbm, lastXs, corrs_train_lgbm = procedure(template_lgbm)
corrs_test_lgbm = test(models_lgbm)

In [ ]:
%%time
# Test de l'interface
lastX = lastXs[0]
# lastX = lastX.shift(-1)
# lastX.iloc[-1] = df_train[df_train["Asset_ID"] == 0].iloc[-1]

x = get_features(lastX, df_btc)
feature_names = x.columns
y_pred = models_lgbm[0].predict(x[-1:])[0]
y_pred

In [ ]:
plot_corrs(corrs_train_lgbm, corrs_test_lgbm)

In [ ]:
# Précédemment meilleur avec plus de caractéristiques
# plot_corrs(corrs_train_lgbm, corrs_test_lgbm)

In [ ]:
import matplotlib.pyplot as plt
ID = 9
plt.figure(figsize=(15,3))
plt.plot(df_test.loc[df_test['Asset_ID'] == ID, 'Target'].values, label='Réel')
plt.plot(df_test.loc[df_test['Asset_ID'] == ID, 'Pred'].values*(3), label='Prédiction')
plt.xlabel('Minutes')
plt.ylabel('Cible')
plt.plot([0,199],[0,0],'k--')
plt.xlim(0,199)
plt.legend();

In [ ]:
plot_feature_importance(feature_names, models_lgbm[9]['model'])

## XGBRegressor

In [ ]:
%%time
models_xgb, lastXs, corrs_train_xgb = procedure(template_xgb)
corrs_test_xgb = test(models_xgb)

In [ ]:
%%time
# Test de l'interface
lastX = lastXs[0]
# lastX = lastX.shift(-1)
# lastX.iloc[-1] = df_train[df_train["Asset_ID"] == 0].iloc[-1]

x = get_features(lastX, df_btc)
feature_names = x.columns
y_pred = models_xgb[0].predict(x[-1:])[0]
y_pred

In [ ]:
plot_corrs(corrs_train_xgb, corrs_test_xgb)

In [ ]:
import matplotlib.pyplot as plt
ID = 9
plt.figure(figsize=(15,3))
plt.plot(df_test.loc[df_test['Asset_ID'] == ID, 'Target'].values, label='Réel')
plt.plot(df_test.loc[df_test['Asset_ID'] == ID, 'Pred'].values*(2), label='Prédiction')
plt.xlabel('Minutes')
plt.ylabel('Cible')
plt.plot([0,199],[0,0],'k--')
plt.xlim(0,199)
plt.legend();

In [ ]:
plot_feature_importance(feature_names, models_xgb[9]['model'])

## Comparaison

In [ ]:
pd.DataFrame(
    data={
        'Entrainement': [
            ponderation(corrs_train_lin),
            ponderation(corrs_train_lgbm),
            ponderation(corrs_train_xgb)],
        'Test': [
            ponderation(corrs_test_lin),
            ponderation(corrs_test_lgbm),
            ponderation(corrs_test_xgb)]},
    index=['Linéaire','LGBM','XGBoost']
).plot.bar(rot=0)
plt.ylabel('Corrélation de Pearson');

---
# Sousmission

Il s'agit d'un concours de code, dans lequel nous devons soumettre une Notebook pour qu'elle soit exécuté avec les données privées cachées. 
La Notebook doit utiliser l'API de série temporelle python fournie, qui garantit que les modèles n'apparaissent pas dans le temps. 
Pour utiliser l'API, les instructions et l'exemple sont détaillées ici : [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction) et [Basic Submission Template](https://www.kaggle.com/sohier/basic-submission-template).

In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
models = models_lgbm
Xs = dict(lastXs)
for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        ID = row['Asset_ID']
        model = models[ID]
        Xs[ID] = Xs[ID].shift(-1)
        Xs[ID].iloc[-1] = row[1:]
        
        x_test = get_features(Xs[ID])
        y_pred = model.predict(x_test[-1:])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
#         # Print just one sample row to get a feeling of what it looks like
#         if i == 0 and j == 0:
#             display(x_test)

#     # Display the first prediction dataframe
#     if i == 0:
#         display(df_pred)

    # Send submissions
    env.predict(df_pred)